<a href="https://colab.research.google.com/github/Yixuan-042/CASA0025/blob/main/notebooks/W04_week4_postgis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [21]:
# %pip install duckdb leafmap lonboard
import duckdb
import leafmap

In [20]:
%pip install duckdb duckdb-engine jupysql

In [22]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [23]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [24]:
%sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db

In [25]:
%%sql

SELECT * FROM duckdb_extensions();

,extension_name,loaded,installed,install_path,description,aliases,extension_version,install_mode,installed_from
0,arrow,False,False,,A zero-copy data integration between Apache Ar...,[],,None,
1,autocomplete,False,False,,Adds support for autocomplete in the shell,[],,None,
2,aws,False,False,,Provides features that depend on the AWS SDK,[],,None,
3,azure,False,False,,Adds a filesystem abstraction for Azure blob s...,[],,None,
4,delta,False,False,,Adds support for Delta Lake,[],,None,
5,excel,False,False,,Adds support for Excel-like format strings,[],,None,
6,fts,False,True,(BUILT-IN),Adds support for Full-Text Search Indexes,[],,STATICALLY_LINKED,
7,httpfs,False,False,,Adds support for reading and writing files ove...,"[http, https, s3]",,None,
8,iceberg,False,False,,Adds support for Apache Iceberg,[],,None,
9,icu,True,True,(BUILT-IN),Adds support for time zones and collations usi...,[],,STATICALLY_LINKED,


In [26]:
%%sql

INSTALL httpfs;
LOAD httpfs;

,Success


In [28]:
import os
import zipfile
import requests

# 下载 ZIP 文件
url = "https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip"
zip_path = "nyc_data.zip"
extract_path = "nyc_data"

# 下载文件
response = requests.get(url)
with open(zip_path, "wb") as f:
    f.write(response.content)

# 解压 ZIP 文件
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to: {extract_path}")


Files extracted to: nyc_data


In [30]:
import os

# 检查解压目录中的文件
extracted_files = os.listdir(extract_path)
print(f"Files in '{extract_path}': {extracted_files}")


Files in 'nyc_data': ['nyc_streets.shp', 'nyc_streets.dbf', 'nyc_census_blocks.dbf', 'nyc_census_blocks.shp', 'nyc_homicides.dbf', 'nyc_neighborhoods.shx', 'nyc_subway_stations.prj', 'nyc_neighborhoods.prj', 'nyc_subway_stations.dbf', 'nyc_streets.prj', 'nyc_census_blocks.shx', 'nyc_census_sociodata.sql', 'nyc_homicides.shx', 'README.txt', 'nyc_homicides.prj', 'nyc_census_blocks.prj', 'nyc_neighborhoods.shp', 'nyc_subway_stations.shp', 'nyc_subway_stations.shx', 'nyc_homicides.shp', 'nyc_streets.shx', 'nyc_neighborhoods.dbf']


In [31]:
pip install geopandas


In [32]:
import geopandas as gpd

# 定义文件路径
data_dir = "nyc_data/"

# 读取各个 Shapefile 文件
census_blocks = gpd.read_file(f"{data_dir}nyc_census_blocks.shp")
homicides = gpd.read_file(f"{data_dir}nyc_homicides.shp")
neighborhoods = gpd.read_file(f"{data_dir}nyc_neighborhoods.shp")
streets = gpd.read_file(f"{data_dir}nyc_streets.shp")
subway_stations = gpd.read_file(f"{data_dir}nyc_subway_stations.shp")

# 查看数据
print("Census Blocks:")
print(census_blocks.head())

print("\nHomicides:")
print(homicides.head())

print("\nNeighborhoods:")
print(neighborhoods.head())

print("\nStreets:")
print(streets.head())

print("\nSubway Stations:")
print(subway_stations.head())


Census Blocks:
             BLKID  POPN_TOTAL  POPN_WHITE  POPN_BLACK  POPN_NATIV  \
0  360850009001000          97          51          32           1   
1  360850020011000          66          52           2           0   
2  360850040001000          62          14          18           2   
3  360850074001000         137          92          12           0   
4  360850096011000         289         230           0           0   

   POPN_ASIAN  POPN_OTHER       BORONAME  \
0           5           8  Staten Island   
1           7           5  Staten Island   
2          25           3  Staten Island   
3          13          20  Staten Island   
4          32          27  Staten Island   

                                            geometry  
0  POLYGON ((577856.547 4499583.235, 577862.635 4...  
1  POLYGON ((578620.717 4495974.818, 578535.358 4...  
2  POLYGON ((577227.224 4495995.067, 577155.625 4...  
3  POLYGON ((579037.033 4494421.77, 579000.015 44...  
4  POLYGON ((577652.483 

In [33]:
# 将 GeoDataFrame 保存为 CSV
census_blocks.to_csv(f"{data_dir}nyc_census_blocks.csv", index=False)
homicides.to_csv(f"{data_dir}nyc_homicides.csv", index=False)
neighborhoods.to_csv(f"{data_dir}nyc_neighborhoods.csv", index=False)
streets.to_csv(f"{data_dir}nyc_streets.csv", index=False)
subway_stations.to_csv(f"{data_dir}nyc_subway_stations.csv", index=False)

print("Files converted to CSV.")


Files converted to CSV.


In [34]:
import duckdb

# 创建 DuckDB 数据库连接
conn = duckdb.connect("nyc_data.duckdb")

# 定义 CSV 文件路径
datasets = {
    "nyc_census_blocks": f"{data_dir}nyc_census_blocks.csv",
    "nyc_homicides": f"{data_dir}nyc_homicides.csv",
    "nyc_neighborhoods": f"{data_dir}nyc_neighborhoods.csv",
    "nyc_streets": f"{data_dir}nyc_streets.csv",
    "nyc_subway_stations": f"{data_dir}nyc_subway_stations.csv",
}

# 加载 CSV 文件到 DuckDB 表
for table_name, file_path in datasets.items():
    print(f"Loading file: {file_path}")
    conn.execute(f"""
        CREATE TABLE {table_name} AS
        SELECT * FROM read_csv_auto('{file_path}');
    """)
    print(f"Table {table_name} created successfully.")

# 检查表是否成功创建
tables = conn.execute("SHOW TABLES;").fetchall()
print("Tables in database:", tables)

# 查询示例
result = conn.execute("SELECT * FROM nyc_neighborhoods LIMIT 5;").fetchdf()
print(result)


Loading file: nyc_data/nyc_census_blocks.csv
Table nyc_census_blocks created successfully.
Loading file: nyc_data/nyc_homicides.csv
Table nyc_homicides created successfully.
Loading file: nyc_data/nyc_neighborhoods.csv
Table nyc_neighborhoods created successfully.
Loading file: nyc_data/nyc_streets.csv
Table nyc_streets created successfully.
Loading file: nyc_data/nyc_subway_stations.csv
Table nyc_subway_stations created successfully.
Tables in database: [('nyc_census_blocks',), ('nyc_homicides',), ('nyc_neighborhoods',), ('nyc_streets',), ('nyc_subway_stations',)]
    BORONAME                      NAME  \
0   Brooklyn               Bensonhurst   
1  Manhattan              East Village   
2  Manhattan              West Village   
3  The Bronx              Throggs Neck   
4  The Bronx  Wakefield-Williamsbridge   

                                            geometry  
0  POLYGON ((582771.4257198056 4495167.427365481,...  
1  POLYGON ((585508.7534890148 4509691.267208001,...  
2  POLYGON

In [36]:
import duckdb

# 连接到数据库
conn = duckdb.connect("nyc_data.duckdb")

# 查看所有表
tables = conn.execute("SHOW TABLES;").fetchall()
print("Available tables:", tables)


Available tables: [('nyc_census_blocks',), ('nyc_homicides',), ('nyc_neighborhoods',), ('nyc_streets',), ('nyc_subway_stations',)]


In [39]:
%%sql
INSTALL spatial;
LOAD spatial;


,Success


In [41]:
%%sql
SELECT * FROM duckdb_extensions();



,extension_name,loaded,installed,install_path,description,aliases,extension_version,install_mode,installed_from
0,arrow,False,False,,A zero-copy data integration between Apache Ar...,[],,None,
1,autocomplete,False,False,,Adds support for autocomplete in the shell,[],,None,
2,aws,False,False,,Provides features that depend on the AWS SDK,[],,None,
3,azure,False,False,,Adds a filesystem abstraction for Azure blob s...,[],,None,
4,delta,False,False,,Adds support for Delta Lake,[],,None,
5,excel,False,False,,Adds support for Excel-like format strings,[],,None,
6,fts,False,True,(BUILT-IN),Adds support for Full-Text Search Indexes,[],,STATICALLY_LINKED,
7,httpfs,True,True,/root/.duckdb/extensions/v1.1.3/linux_amd64_gc...,Adds support for reading and writing files ove...,"[http, https, s3]",v1.1.3,REPOSITORY,core
8,iceberg,False,False,,Adds support for Apache Iceberg,[],,None,
9,icu,True,True,(BUILT-IN),Adds support for time zones and collations usi...,[],,STATICALLY_LINKED,


In [42]:
%%sql
SELECT ST_Contains(ST_GeomFromText('POLYGON((0 0, 0 1, 1 1, 1 0, 0 0))'), ST_GeomFromText('POINT(0.5 0.5)')) AS contains_result;


,contains_result
0,True


In [47]:
# 列出当前目录下的所有文件
!ls


nyc_data  nyc_data.db  nyc_data.duckdb	nyc_data.duckdb.wal  nyc_data.zip  sample_data


In [48]:
!ls nyc_data/


nyc_census_blocks.csv	  nyc_homicides.dbf	 nyc_neighborhoods.shp	nyc_subway_stations.csv
nyc_census_blocks.dbf	  nyc_homicides.prj	 nyc_neighborhoods.shx	nyc_subway_stations.dbf
nyc_census_blocks.prj	  nyc_homicides.shp	 nyc_streets.csv	nyc_subway_stations.prj
nyc_census_blocks.shp	  nyc_homicides.shx	 nyc_streets.dbf	nyc_subway_stations.shp
nyc_census_blocks.shx	  nyc_neighborhoods.csv  nyc_streets.prj	nyc_subway_stations.shx
nyc_census_sociodata.sql  nyc_neighborhoods.dbf  nyc_streets.shp	README.txt
nyc_homicides.csv	  nyc_neighborhoods.prj  nyc_streets.shx


In [49]:
%%sql
CREATE TABLE nyc_neighborhoods AS SELECT * FROM read_csv_auto('/content/nyc_data/nyc_neighborhoods.csv');

,Success


In [51]:
%%sql
CREATE TABLE nyc_census_blocks AS SELECT * FROM read_csv_auto('/content/nyc_data/nyc_census_blocks.csv');

,Success


In [52]:
%%sql
CREATE TABLE nyc_homicides AS SELECT * FROM read_csv_auto('/content/nyc_data/nyc_homicides.csv');

,Success


In [53]:
%%sql
CREATE TABLE nyc_streets AS SELECT * FROM read_csv_auto('/content/nyc_data/nyc_streets.csv');

,Success


In [54]:
%%sql
CREATE TABLE nyc_subway_stations AS SELECT * FROM read_csv_auto('/content/nyc_data/nyc_subway_stations.csv');

,Success


In [57]:
%%sql

FROM nyc_census_blocks;

,BLKID,POPN_TOTAL,POPN_WHITE,POPN_BLACK,POPN_NATIV,POPN_ASIAN,POPN_OTHER,BORONAME,geometry
0,360850009001000,97,51,32,1,5,8,Staten Island,"POLYGON ((577856.5470479821 4499583.234929237,..."
1,360850020011000,66,52,2,0,7,5,Staten Island,"POLYGON ((578620.7173632095 4495974.817866362,..."
2,360850040001000,62,14,18,2,25,3,Staten Island,"POLYGON ((577227.2244709881 4495995.066845497,..."
3,360850074001000,137,92,12,0,13,20,Staten Island,"POLYGON ((579037.0332016965 4494421.769816227,..."
4,360850096011000,289,230,0,0,32,27,Staten Island,"POLYGON ((577652.4825280879 4494975.052285533,..."
...,...,...,...,...,...,...,...,...,...
38789,360050295001004,328,267,14,2,8,37,The Bronx,"POLYGON ((592015.6683484344 4526716.91039808, ..."
38790,360050295002002,0,0,0,0,0,0,The Bronx,"POLYGON ((592141.9363578608 4526852.139093006,..."
38791,360050419004001,0,0,0,0,0,0,The Bronx,"POLYGON ((594123.2874226582 4525906.39647421, ..."
38792,360050255002001,480,96,96,20,12,256,The Bronx,"POLYGON ((591982.369936439 4523793.5686508985,..."


In [58]:
%%sql

FROM nyc_homicides;

,INCIDENT_D,BORONAME,NUM_VICTIM,PRIMARY_MO,ID,WEAPON,LIGHT_DARK,YEAR,geometry
0,2008-01-01,Brooklyn,1.0,None,7,gun,D,2008,POINT (592158.6657641566 4502210.892367315)
1,2008-01-04,Manhattan,1.0,None,14,gun,D,2008,POINT (588654.9516122746 4517855.382656676)
2,2008-01-05,Queens,1.0,None,15,gun,D,2008,POINT (605800.8150245796 4505730.608395767)
3,2008-01-04,Queens,1.0,None,16,knife,D,2008,POINT (594255.1571757384 4512250.378115875)
4,2008-01-05,Queens,1.0,None,18,gun,D,2008,POINT (605498.1350349792 4496052.6404242935)
...,...,...,...,...,...,...,...,...,...
3977,2010-10-11,The Bronx,1.0,None,4269,gun,None,2010,POINT (592515.9578506276 4523492.764356317)
3978,2010-10-06,The Bronx,1.0,None,4271,knife,None,2010,POINT (594347.5429990037 4522235.809302402)
3979,2011-07-26,The Bronx,1.0,None,4282,gun,None,2011,POINT (592849.08922107 4522545.555276063)
3980,2011-07-28,The Bronx,1.0,None,4284,gun,None,2011,POINT (596872.5868381617 4528682.876480885)


In [59]:
%%sql

FROM nyc_neighborhoods;

,BORONAME,NAME,geometry
0,Brooklyn,Bensonhurst,"POLYGON ((582771.4257198056 4495167.427365481,..."
1,Manhattan,East Village,"POLYGON ((585508.7534890148 4509691.267208001,..."
2,Manhattan,West Village,"POLYGON ((583263.2776595836 4509242.626023987,..."
3,The Bronx,Throggs Neck,"POLYGON ((597640.0090688139 4520272.719938631,..."
4,The Bronx,Wakefield-Williamsbridge,"POLYGON ((595285.2053417757 4525938.79838847, ..."
...,...,...,...
124,Brooklyn,Red Hook,MULTIPOLYGON (((584212.898102789 4502321.47444...
125,Queens,Douglastown-Little Neck,"POLYGON ((605082.2876993549 4513540.148431633,..."
126,Queens,Whitestone,MULTIPOLYGON (((600138.4932375996 4516909.4994...
127,Queens,Steinway,MULTIPOLYGON (((593231.5525230656 4515088.5390...


In [60]:
%%sql

FROM nyc_streets;

,ID,NAME,ONEWAY,TYPE,geometry
0,1,Shore Pky S,None,residential,LINESTRING (586785.4767897038 4492901.00145547...
1,2,None,None,footway,LINESTRING (586645.0073625665 4504977.75036058...
2,3,Avenue O,None,residential,LINESTRING (586750.3019977848 4496109.72213903...
3,4,Walsh Ct,None,residential,"LINESTRING (586728.695515043 4497971.05313857,..."
4,5,None,None,motorway_link,LINESTRING (586587.0531467082 4510088.25040298...
...,...,...,...,...,...
19086,17378,FDR Dr,yes,motorway,LINESTRING (585750.4886479316 4507129.54674604...
19087,17382,FDR Dr,yes,motorway,LINESTRING (583473.1273160246 4506080.00100516...
19088,19039,FDR Dr,None,motorway,LINESTRING (586815.6257745615 4509044.98349808...
19089,19053,FDR Dr,None,motorway,LINESTRING (586559.0172856033 4509871.20762023...


In [61]:
%%sql

FROM nyc_subway_stations;

,OBJECTID,ID,NAME,ALT_NAME,CROSS_ST,LONG_NAME,LABEL,BOROUGH,NGHBHD,ROUTES,TRANSFERS,COLOR,EXPRESS,CLOSED,geometry
0,1.0,376.0,Cortlandt St,None,Church St,"Cortlandt St (R,W) Manhattan","Cortlandt St (R,W)",Manhattan,None,"R,W","R,W",YELLOW,None,NaN,POINT (583521.854408956 4507077.862599085)
1,2.0,2.0,Rector St,None,None,Rector St (1) Manhattan,Rector St (1),Manhattan,None,1,1,RED,None,NaN,POINT (583324.4866324601 4506805.373160211)
2,3.0,1.0,South Ferry,None,None,South Ferry (1) Manhattan,South Ferry (1),Manhattan,None,1,1,RED,None,NaN,POINT (583304.1823994748 4506069.654048115)
3,4.0,125.0,138th St,Grand Concourse,Grand Concourse,"138th St / Grand Concourse (4,5) Bronx","138th St / Grand Concourse (4,5)",Bronx,None,"4,5","4,5",GREEN,None,NaN,POINT (590250.10594797 4518558.019924332)
4,5.0,126.0,149th St,Grand Concourse,Grand Concourse,149th St / Grand Concourse (4) Bronx,149th St / Grand Concourse (4),Bronx,None,4,"2,4,5",GREEN,express,NaN,POINT (590454.7399891173 4519145.719617855)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,487.0,909.0,JFK Terminal 8,None,None,"JFK Terminal 8, Queens",JFK Terminal 8,Queens,None,None,None,AIR-BLUE,None,NaN,POINT (602433.3533596311 4500363.8128236225)
487,488.0,903.0,Federal Circle,Rental Car,None,"Federal Circle / Rental Car, Queens",Federal Circle / Rental Car,Queens,None,None,None,AIR-BLUE,None,NaN,POINT (600903.1428474564 4501689.113908147)
488,489.0,902.0,Long Term Parking,None,None,"Long Term Parking, Queens",Long Term Parking,Queens,None,None,None,AIR-BLUE,None,NaN,POINT (599552.1129832724 4502056.229470152)
489,490.0,901.0,Howard Beach,None,159th Ave,"Howard Beach, Queens",Howard Beach,Queens,None,None,A,AIR-BLUE,None,NaN,POINT (598862.0205107569 4501868.071868393)


Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [65]:
%%sql
-- 查看 nyc_neighborhoods 表的前几行数据
SELECT * FROM nyc_neighborhoods LIMIT 5;

-- 查看 nyc_subway_stations 表的前几行数据
SELECT * FROM nyc_subway_stations LIMIT 5;



,OBJECTID,ID,NAME,ALT_NAME,CROSS_ST,LONG_NAME,LABEL,BOROUGH,NGHBHD,ROUTES,TRANSFERS,COLOR,EXPRESS,CLOSED,geometry
0,1.0,376.0,Cortlandt St,None,Church St,"Cortlandt St (R,W) Manhattan","Cortlandt St (R,W)",Manhattan,None,"R,W","R,W",YELLOW,None,NaN,POINT (583521.854408956 4507077.862599085)
1,2.0,2.0,Rector St,None,None,Rector St (1) Manhattan,Rector St (1),Manhattan,None,1,1,RED,None,NaN,POINT (583324.4866324601 4506805.373160211)
2,3.0,1.0,South Ferry,None,None,South Ferry (1) Manhattan,South Ferry (1),Manhattan,None,1,1,RED,None,NaN,POINT (583304.1823994748 4506069.654048115)
3,4.0,125.0,138th St,Grand Concourse,Grand Concourse,"138th St / Grand Concourse (4,5) Bronx","138th St / Grand Concourse (4,5)",Bronx,None,"4,5","4,5",GREEN,None,NaN,POINT (590250.10594797 4518558.019924332)
4,5.0,126.0,149th St,Grand Concourse,Grand Concourse,149th St / Grand Concourse (4) Bronx,149th St / Grand Concourse (4),Bronx,None,4,"2,4,5",GREEN,express,NaN,POINT (590454.7399891173 4519145.719617855)


In [72]:
%%sql
SELECT s.name AS station_name, s.routes AS subway_routes
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods AS n
ON ST_Contains(ST_GeomFromText(n.geometry), ST_GeomFromText(s.geometry))  -- 转换为几何类型
WHERE n.name = 'Little Italy';


,station_name,subway_routes
0,Spring St,6


2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [82]:
%%sql
SELECT DISTINCT n.name AS station_name, n.boroname AS subway_routes
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods AS n
ON ST_Contains(ST_GeomFromText(n.geometry), ST_GeomFromText(s.geometry))
WHERE strpos(s.routes,'6') > 0;


,station_name,subway_routes
0,Financial District,Manhattan
1,Little Italy,Manhattan
2,Upper East Side,Manhattan
3,East Harlem,Manhattan
4,Mott Haven,The Bronx
5,Hunts Point,The Bronx
6,South Bronx,The Bronx
7,Chinatown,Manhattan
8,Greenwich Village,Manhattan
9,Gramercy,Manhattan


3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [86]:
%%sql
SELECT SUM(popn_total)
FROM nyc_neighborhoods AS n
JOIN nyc_census_blocks AS c
ON ST_INTERSECTS(ST_GeomFromText(n.geometry), ST_GeomFromText(c.geometry))
WHERE n.name = 'Battery Park';

,sum(popn_total)
0,17153.0


4. **What neighborhood has the highest population density (persons/km2)?**


In [90]:
%%sql
SELECT n.name,
sum(c.popn_total)/ST_Area(ST_GeomFromText(n.geometry))/1000000.0 AS popn_per_skqm
FROM nyc_census_blocks AS c
JOIN nyc_neighborhoods AS n
ON ST_INTERSECTS(ST_GeomFromText(c.geometry), ST_GeomFromText(n.geometry))
GROUP BY n.name, n.geometry
ORDER BY popn_per_skqm DESC
LIMIT 1;

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,NAME,popn_per_skqm
0,North Sutton Area,6.843513e-08


When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [91]:
%pip install duckdb duckdb-engine jupysql

In [92]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [93]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

,Success


## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

In [102]:
%%sql
-- 创建一个 SEQUENCE，用于生成自增 ID
CREATE SEQUENCE ais_id_seq;

-- 创建表，并使用 SEQUENCE 生成自增主键
CREATE TABLE ais (
    id INTEGER DEFAULT nextval('ais_id_seq'), -- 自动递增主键
    vesselid INT NOT NULL,                    -- 船只唯一标识符
    sog FLOAT,                                -- 地面速度（节）
    draught FLOAT,                            -- 吃水深度（米）
    date TIMESTAMP,                           -- AIS 信号时间戳
    geom TEXT                                 -- 几何列，存储为 WKT 格式
);


,Success


In [103]:
%%sql
INSERT INTO ais (vesselid, sog, draught, date, geom)
VALUES (1, 12.5, 7.2, '2025-02-07 18:00:00', 'POINT(30 10)');

INSERT INTO ais (vesselid, sog, draught, date, geom)
VALUES (2, 15.3, 8.1, '2025-02-07 19:00:00', 'POINT(40 20)');

-- 查看数据
SELECT * FROM ais;


,id,vesselid,sog,draught,date,geom
0,1,1,12.5,7.2,2025-02-07 18:00:00,POINT(30 10)
1,2,2,15.3,8.1,2025-02-07 19:00:00,POINT(40 20)


In [104]:
%%sql
CREATE TABLE vinfo (
    vesselid INT PRIMARY KEY,             -- 船只唯一标识符
    vessel_name TEXT,                     -- 船只名称
    vsl_descr TEXT,                       -- 船只类型
    dwt FLOAT,                            -- 载重吨位
    v_length FLOAT                        -- 船只长度（米）
);


,Success


In [108]:
pip install --upgrade duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 69.9 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.1.3
    Uninstalling duckdb-1.1.3:
      Successfully uninstalled duckdb-1.1.3


In [107]:
%%sql
INSTALL httpfs;

LOAD httpfs;

COPY ais (vesselid, sog, draught, date, geom)
FROM 'https://storage.googleapis.com/path/to/casa0025_ships.csv'
DELIMITER ',' CSV HEADER;



RuntimeError: (duckdb.duckdb.TransactionException) TransactionContext Error: Current transaction is aborted (please ROLLBACK)
[SQL: INSTALL httpfs;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
If you need help solving this issue, send us a message: https://ploomber.io/community


## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.